In [2]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.keys import Keys
from webdriver_manager.chrome import ChromeDriverManager
import time


In [3]:
# Function to read ingredients from external file
def read_ingredients(file_path):
    try:
        with open(file_path, 'r') as file:
            # Read all lines from the file and split into tuples (name, SKU)
            ingredients = [tuple(line.strip().split(',')) for line in file.readlines()]
        return ingredients
    except FileNotFoundError:
        print(f"Error: The file {file_path} was not found.")
        return []

In [4]:
# Function to generate the report
def generate_report(added_items, not_found_items):
    print("\n--- Shopping Report ---")
    print(f"Items successfully added to the cart: {len(added_items)}")
    for item in added_items:
        print(f"  - {item}")

    print(f"\nItems not found: {len(not_found_items)}")
    for item in not_found_items:
        print(f"  - {item}")

In [5]:
# Set up the WebDriver
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))

# Open Sainsbury's website
driver.get("https://www.sainsburys.co.uk")

# Wait for the cookies banner to appear
time.sleep(5)

# Find and click the "Accept" button on the cookies banner
try:
    accept_button = driver.find_element(By.XPATH, '//*[@id="onetrust-accept-btn-handler"]')  # Example XPath
    accept_button.click()
except Exception as e:
    print("Cookies banner not found or already accepted")

# Find and search the search bar to reach the next page (so ingredients loop can use 1 search bar reference)
try:
    # Locate the search bar (update the By selector as needed)
    search_bar = driver.find_element(By.XPATH,'//*[@id="term"]')  # Example ID, replace with actual ID or selector

    # Clear the search bar in case anything is already there
    search_bar.clear()

    # Enter the ingredient in the search bar (doesn't matter what, just move to next page)
    search_bar.send_keys("potato")

    # Simulate pressing Enter
    search_bar.send_keys(Keys.RETURN)

    # Wait for the results page to load
    time.sleep(5)

    # Click add potato to navigate to login screen (more consistent than login button)
    add_button = driver.find_element(By.XPATH, "//button[contains(@class, 'add-button')]")
    add_button.click()  # Click the "Add" button

except Exception as e:
    print(f"An error occurred while searching")


In [6]:
# Try to log in before we start on the list
try:
    # Use a generalized XPath or CSS Selector to locate the "Log in" button (temp: try just clicking add potato to get here instead)
    #login_button = driver.find_element(By.css_selector, "[data-testid='login-register-link']")
    #login_button.click()  # Click the "Add" button

    # Wait for the results page to load
    time.sleep(5)

    # Find and click the "Accept" button on the cookies banner
    accept_button = driver.find_element(By.XPATH, '//*[@id="onetrust-accept-btn-handler"]')  # Example XPath
    accept_button.click()

    # Locate the username bar (update the By selector as needed)
    search_bar = driver.find_element(By.XPATH,'//*[@id="username"]')

    # Enter the username
    search_bar.send_keys("dbryant1995@hotmail.com")

    # Locate the pw bar (update the By selector as needed)
    search_bar = driver.find_element(By.XPATH,'//*[@id="password"]')

    # Enter the pw
    search_bar.send_keys("Password") # pw goes here

    # Simulate pressing Enter
    search_bar.send_keys(Keys.RETURN)

    # Wait for the results page to load
    time.sleep(3)
    print(f"Successfully logged in")

except Exception as e:
    print(f"Could not log in")


Successfully logged in


In [7]:
# Specify the path to your ingredients file
file_path = 'C:\\Users\\dbrya\\Documents\\ingredients.txt' # ammend path as required

# Read ingredients from the file
ingredients = read_ingredients(file_path)

# Wait for the page to load
time.sleep(5)

# Lists to store results for the report
added_items = []
not_found_items = []


In [8]:
# Loop through each ingredient (name, SKU) and perform a search
for ingredient_name, sku in ingredients:
    try:
        # Locate the search bar
        search_bar = driver.find_element(By.XPATH,'//*[@id="search-bar-input"]')

        # Clear the search bar
        search_bar.send_keys(Keys.CONTROL + "a")  # Select all text
        search_bar.send_keys(Keys.BACKSPACE)  # Delete selected text

        # Enter the sku in the search bar
        search_bar.send_keys(sku)

        # Simulate pressing Enter
        search_bar.send_keys(Keys.RETURN)

        # Wait for the results page to load
        time.sleep(3)

        # Find the first "Add" button for the search result
        try:
            add_button = driver.find_element(By.XPATH, "//button[contains(@class, 'add-button')]")
            add_button.click()  # Click the "Add" button
            print(f"Successfully added {ingredient_name} to the cart.")
            added_items.append(ingredient_name)  # Add to the list of successfully added items

        except Exception as e:
            print(f"Could not find an 'Add' button for {ingredient_name}.")
            not_found_items.append(ingredient_name)  # Add to the list of items not found

    except Exception as e:
        print(f"An error occurred while searching for {ingredient_name}: {e}")
        not_found_items.append(ingredient_name)  # Add to the list of items not found


Successfully added Chicken thighs to the cart.
Successfully added Onions to the cart.
Successfully added Fresh ginger to the cart.
Successfully added Fresh coriander to the cart.
Successfully added fresh tomatoes to the cart.
Successfully added Greek yoghurt to the cart.
Successfully added Creme fraiche to the cart.
Successfully added Quorn chicken to the cart.
Successfully added coconut milk to the cart.
Could not find an 'Add' button for coconut milk.
Successfully added Bamboo shoots to the cart.
Successfully added Peppers to the cart.
Successfully added Chestnut mushrooms to the cart.
Could not find an 'Add' button for Chestnut mushrooms.
Successfully added Chillies to the cart.
Successfully added Garlic to the cart.
Successfully added Portobello mushrooms to the cart.
Successfully added White mushrooms to the cart.
Successfully added Fresh parsley to the cart.
Successfully added Soured cream to the cart.
Successfully added Burgers to the cart.
Successfully added Buns to the cart.
S

In [9]:
# Generate the report
generate_report(added_items, not_found_items)

# Some time after the script runs to inspect page
time.sleep(50)

# Close the browser after all searches are complete
driver.quit()


--- Shopping Report ---
Items successfully added to the cart: 39
  - Chicken thighs
  - Onions
  - Fresh ginger
  - Fresh coriander
  - fresh tomatoes
  - Greek yoghurt
  - Creme fraiche
  - Quorn chicken
  - coconut milk
  - Bamboo shoots
  - Peppers
  - Chestnut mushrooms
  - Chillies
  - Garlic
  - Portobello mushrooms
  - White mushrooms
  - Fresh parsley
  - Soured cream
  - Burgers
  - Buns
  - Gluten free buns
  - Sweet potatoes
  - White cabbage
  - Carrots
  - Spinach
  - Bacon
  - Kimchi
  - Mackerel
  - More mackerel
  - Baby potatoes
  - Spring onions
  - Fresh basil
  - Cucumber
  - Radishes
  - Pomegranate seeds
  - Cottage cheese
  - Jumbo oats
  - Blueberries
  - Fresh lemons

Items not found: 4
  - coconut milk
  - Chestnut mushrooms
  - Sweet potatoes
  - Carrots
